<image src = "https://raw.githubusercontent.com/panaverse/learn-generative-ai/94f3bb6966ce89d09ce8998e0d26425a4eb7c3ed/03_chatgpt/07_assistants_knowledge_retrieval/ret.jpg">

# Pretty Printing Helper 

In [49]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [11]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv
import os


load_dotenv(find_dotenv())

openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

client: openai = OpenAI()


# Step 1:
* uploaing a file and creating an assistant 

In [52]:
from openai.types.beta import Assistant

# Upload a file with an "assistants" purpose
file = client.files.create(
  file=open("zia_profile.pdf", "rb"),
  purpose='assistants'
)

show_json(file)

{'id': 'file-mzfXjY4t9i2KlgfK5bZcmgJq',
 'bytes': 48802,
 'created_at': 1706893623,
 'filename': 'zia_profile.pdf',
 'object': 'file',
 'purpose': 'assistants',
 'status': 'processed',
 'status_details': None}

In [25]:
assistant: Assistant = client.beta.assistants.create(
  name="Student Support Assistant",
  instructions="You are a student support chatbot. Use your knowledge base to best respond to student queries about Zia  Khan.",
  model="gpt-3.5-turbo-1106",
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)

# Step 2
* Creating a Thread 

In [53]:
from openai.types.beta.thread import Thread

thread : Thread = client.beta.threads.create()

show_json (thread)

{'id': 'thread_hOmnHMfYGe4qhYvJa4YPQMC6',
 'created_at': 1706893638,
 'metadata': {},
 'object': 'thread'}

# Step 3 
* Create the Message 

In [28]:
from openai.types.beta.threads.thread_message import ThreadMessage 

message = client.beta.threads.messages.create(
    thread_id= thread.id,
    role = 'user',
    content = "When and which city was zia born"

)

# Step 4
* Run the message

In [54]:
from openai.types.beta.threads.run import Run

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions = "the user is one of the student from PIAIC"
)

show_json (run)

{'id': 'run_9FtEEh0wBbG80oO0QNeJLN7z',
 'assistant_id': 'asst_pflxHRN5WxOqPNRsKtxvuA07',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1706893648,
 'expires_at': 1706894248,
 'failed_at': None,
 'file_ids': ['file-UhUsIVFUTND53iAmzsGKOZAO'],
 'instructions': 'the user is one of the student from PIAIC',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_hOmnHMfYGe4qhYvJa4YPQMC6',
 'tools': [{'type': 'retrieval'}],
 'usage': None}

# Step 4
* check the run status 

In [47]:

import time
def time_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run : Run = client.beta.threads.runs.retrieve(
    
    thread_id = thread.id,
    run_id = run.id,
    ) 

        time.sleep(5)
    return run  

       




    



In [55]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_cNAkFhNuTSle8GVKkzZF80fh',
   'assistant_id': 'asst_pflxHRN5WxOqPNRsKtxvuA07',
   'content': [{'text': {'annotations': [{'end_index': 406,
        'file_citation': {'file_id': 'file-UhUsIVFUTND53iAmzsGKOZAO',
         'quote': "A Nation-Building Journey of 40 Years\n\n\nZia Khan was born in Sialkot in 1961 in an army garrison. His father the late Lt.\nCol. Inam Ullah Khan was an army officer who had been posted to Sialkot in an\nartillery regiment. The family moved throughout Pakistan in different positions.\nThe events and tragedy of December 1971 had a deep impact on his thinking and\npsyche. He felt that he had to do something to turn the future of his beloved\nPakistan around. However exactly what needed to be done remained a mystery\nto him.\n\n\nZia completed his matriculation from Habib Public School in Karachi in 1977 in\nthe first division. For the next few years he knew that he would spend his life\nworking for the betterment of his country but the speci

In [56]:
messages: list[ThreadMessage] = client.beta.threads.messages.list(
    thread_id=thread.id
)

for m in reversed(messages.data):
    print( m.role + ":" + m.content[0].text.value)

assistant:Zia U. Khan, the CEO and Founder of Panacloud (Pvt.) Ltd. and Panaverse LLC, as well as the COO and Founder of PIAIC, has a remarkable journey dedicated to nation-building in Pakistan. His efforts have led to the formation of the Presidential Initiative for Artificial Intelligence and Computing (PIAIC), which has trained thousands of students in cutting-edge technologies and continues to do so【6†source】. If you need further information on PIAIC or any other details from the document, feel free to ask!
